This is where the magic happens:

# MODELLING

In this notebook I'll work on a model to what a tourist will spend when vacationing in Tanzania.
The evaluation metric for the model is **Mean Absolute Error**.


To do:
- train test split
- preprocessing (function or pipelines): outliers, missing values, scaling, dummification, encoding etc.
- baseline model
- feature selection/feature engineering (subregions)
- model
- again if necessary: feature selection/feature engineering
- hyperparameter tuning
- interpretation/visualization